<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/final_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install -qU python-dotenv langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb faiss-cpu langchain-classic openai pandas camelot-py[cv] PyMuPDF wikipedia_api sentence-transformers rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00

In [3]:
pip uninstall -y transformers

Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3


In [4]:
pip install --no-cache-dir "transformers>=4.45.0"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 138.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 340.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 366.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.7.5 requires tokenizers<0.21.0,>=0.20.0, but you have tokenizers 0.22.1 which is incompatible.


In [5]:
import os
import re
import json
from collections import Counter
from typing import List, Tuple, Dict, Any

import pandas as pd
import wikipediaapi

from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_classic.chains import RetrievalQA
from langchain_classic.retrievers import EnsembleRetriever
from openai import OpenAI

import camelot
import fitz

from langchain_core.documents import Document
from langchain_upstage import UpstageDocumentParseLoader

import uuid
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_core.stores import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

import collections

import numpy as np
from rank_bm25 import BM25Okapi

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
No module named 'transformers.modeling_layers'
이런 에러뜨면 밑에 트랜스포머 지우고 다시깔기

In [6]:
cd /content/drive/MyDrive/assignment/25-2/NLP

/content/drive/MyDrive/assignment/25-2/NLP


In [7]:
TESTSET_PATH = "./dataset/testset.csv"

UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"

# EWHA 학칙

In [8]:
EWHA_PDF_PATH = "./ewha.pdf"

In [9]:
upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

In [10]:
loader = UpstageDocumentParseLoader(
    EWHA_PDF_PATH,
    api_key=UPSTAGE_API_KEY
)
docs = loader.load()

In [11]:
clean_texts = []

for doc in docs:
    text = doc.page_content

    text = text.strip()
    clean_texts.append(text)


full_text = "\n".join(clean_texts)

In [12]:
match = re.search(r"부칙", full_text)

if not match:
        raise ValueError("부칙 문구를 찾을 수 없습니다.")

addendum_start = match.start()

main_text = text[:addendum_start]
extra_text = text[addendum_start:]

In [13]:
def extract_main_text(html_text):
    # 테이블 삭제
    cleaned = re.sub(r'<table.*?>.*?</table>', '', html_text, flags=re.DOTALL)
    return cleaned

cleaned_extra_text = extract_main_text(extra_text)

In [14]:
client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1"
)

In [15]:
prompt = """
You are an expert in Korean legal document analysis and Markdown formatting.

Your task:
Convert the given Ewha University regulation texts (in HTML text) into a clean Markdown document with correctly assigned heading levels.
You are given a main text:
1) MAIN TEXT: the main university regulations

Process BOTH texts together and apply the same rules.

=================================================================
MARKDOWN HEADING RULES
=================================================================

**Main Body Chapters (e.g., “제1장 총칙”)**:
- Top-level header: `# [Chapter Title]`
- Articles within chapters: `## [Article Title]`

=================================================================
ADDITIONAL RULES
=================================================================
- Remove ALL page numbers (e.g., “2-2-1”).
- Remove HTML tags, styles, tables, or formatting artifacts from the PDF conversion.
- Preserve all meaningful text content, but organize it strictly under the specified Markdown headings.
- Do NOT hallucinate any content not present in the text.
- Do NOT output explanations or any text outside the markdown.
- Output MUST be clean markdown and ONLY markdown.

Below are the two raw texts:

---------------- MAIN TEXT START ----------------
{main_text}
---------------- MAIN TEXT END ----------------

"""


messages = [
    {
        "role": "system",
        "content": "You are an expert in hierarchical legal information extraction and Markdown document structuring."
    },
    {
        "role": "user",
        "content": prompt.format(main_text=main_text)
    }
]

md_response = client.chat.completions.create(
    model="solar-pro2",
    messages=messages
    # response_format=response_format
)

In [16]:
extra_prompt = """
You are an expert in Korean legal document analysis and Markdown formatting.

Your task:
Convert the given text into a clean Markdown document with correctly assigned heading levels.

=================================================================
MARKDOWN HEADING RULES
=================================================================

**Main Body Chapters (e.g., “제1장 총칙”)**:
- Top-level header: `# [Chapter Title]`
- Articles within chapters: `## [Article Title]`

=================================================================
SPECIFIC INSTRUCTIONS FOR THIS TEXT
=================================================================
- The overall title for this section should be `# 이화여자대학교 학칙`. This should be the first line of the output.
- Every instance of '부칙' that introduces a new addendum should be formatted as a second-level Markdown header, '## 부칙'. Ensure its associated content follows immediately under this header.
- Remove ALL page numbers (e.g., “2-2-1”).
- Remove HTML tags, styles, tables, or formatting artifacts from the PDF conversion.
- Preserve all meaningful text content, but organize it strictly under the specified Markdown headings.
- Do NOT hallucinate any content not present in the text.
- Do NOT output explanations or any text outside the markdown.
- Output MUST be clean markdown and ONLY markdown.

Below is the raw text:

---------------- ADDENDUM TEXT START ----------------
{extra_text}
---------------- ADDENDUM TEXT END ----------------
"""

extra_messages = [
    {
        "role": "system",
        "content": "You are an expert in hierarchical legal information extraction and Markdown document structuring."
    },
    {
        "role": "user",
        "content": extra_prompt.format(extra_text=cleaned_extra_text)
    }
]

md_extra_response = client.chat.completions.create(
    model="solar-pro2",
    messages=extra_messages
)

print(md_extra_response.choices[0].message.content[:500])

# 이화여자대학교 학칙

## 부칙(1946. 8. 15 제정)
본 학칙은 1946년 3월 1일부터 시행한다.

## 부칙
① 본 학칙은 1961년 3월 1일부터 시행한다.  
② 본 학칙 시행에 관한 세칙은 총장이 정한다.

## 부칙(문관행 1040.1-1549, 1963. 12. 16 개정)
본 학칙은 1964년 3월 1일부터 시행한다.

## 부칙(문고대 1041.3-16, 1965. 1. 5 개정)
① 이 학칙은 1965년 1월 10일부터 시행한다.  
② 이 학칙 시행 당시 문리과대학에 소속하는 가정학과와 사범대학에 소속하는 가정학과는  
폐과하되 그 재적 학생의 졸업년도까지 존치한다.

## 부칙(문고대 1041.3-89, 1966. 2. 1 개정)
본 학칙은 1966년 2월 1일부터 시행한다.

## 부칙(문고대 1041.3-89, 1966. 2. 1 개정)
본 학칙은 1966년 2월 1일부터 시행한다.

## 부칙(문고대 1041.3-171, 1967. 1. 24 개


In [17]:
headers_to_split_on = [("##", "Header 2")]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)

md_header_splits = markdown_splitter.split_text(md_response.choices[0].message.content)
extra_splits = markdown_splitter.split_text(md_extra_response.choices[0].message.content)
md_header_splits.extend(extra_splits)

In [18]:
small_headers_to_split_on = [
    ("##", "Header 2"),
     ("###", "Header 3")
]
small_md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=small_headers_to_split_on, strip_headers=False)
small_headers_splits = small_md_splitter.split_text(md_response.choices[0].message.content)
small_headers_splits.extend(small_md_splitter.split_text(md_extra_response.choices[0].message.content))

In [19]:
pdf_path = EWHA_PDF_PATH

# Step 1: lattice 모드로 테이블 bbox만 추출
lattice_tables = camelot.read_pdf(pdf_path, pages="all", flavor="lattice")

final_tables = []

for tbl in lattice_tables:
    page_num = tbl.page
    bbox = tbl._bbox  # (x1, y1, x2, y2)

    # Step 2: PyMuPDF로 페이지 crop
    doc = fitz.open(pdf_path)
    page = doc[page_num - 1]

    rect = fitz.Rect(bbox)
    clipped = page.get_pixmap(clip=rect)
    tmp_img = f"tmp_page_{page_num}.png"
    clipped.save(tmp_img)

    # Step 3: stream 모드로 crop된 이미지 분석
    stream_tbls = camelot.read_pdf(
        pdf_path,
        pages=str(page_num),
        flavor="stream",
        table_areas=[",".join(map(str, bbox))]  # bbox로 영역 제한
    )

    # Step 4: 이 테이블들만 최종 저장
    for s in stream_tbls:
        final_tables.append(s.df)

# 출력 예시
for i, df in enumerate(final_tables):
    # print(f"=== Final Table {i+1} ===")
    # print(df.to_markdown(index=False))
    df.to_csv(f'table{i}.csv', index=False)
    print(f'Table {i} extracted')

cleaned_df = []

for _, df in enumerate(final_tables):
  headers_raw = [str(item).strip().replace(' ', '').replace('\n', '').replace('\u3000', '') for item in df.iloc[1]]

  for col in df.columns:
      df[col] = df[col].astype(str).str.strip().str.replace(' ', '').str.replace('\n', '').str.replace('\u3000', '')
      df[col] = df[col].replace('', np.nan)

  cleaned_df.append(df)

for df in cleaned_df[:3]:
  print(df)
  print('==========================')

Table 0 extracted
Table 1 extracted
Table 2 extracted
Table 3 extracted
Table 4 extracted
Table 5 extracted
Table 6 extracted
Table 7 extracted
Table 8 extracted
Table 9 extracted
Table 10 extracted
Table 11 extracted
Table 12 extracted
Table 13 extracted
Table 14 extracted
Table 15 extracted
Table 16 extracted
Table 17 extracted
         0      1         2
0       대학  학위의종류    학과또는전공
1   사회과학대학    문학사     사회복지학
2   자연과학대학    이학사    분자생명과학
3     공과대학    공학사      컴퓨터학
4      NaN    NaN     정보통신학
5      NaN    NaN  건축학(4년제)
6      NaN   건축학사  건축학(5년제)
7      NaN    공학사       환경학
8     음악대학   음악학사      건반악기
9      NaN    NaN       관현악
10     NaN    NaN        성악
11     NaN    NaN      교회음악
12     NaN    NaN        작곡
13     NaN    NaN      한국음악
14  조형예술대학   미술학사       한국화
15     NaN    NaN     회화․판화
16     NaN    NaN        조소
17     NaN    NaN     환경디자인
18     NaN    NaN   시각정보디자인
19     NaN    NaN     산업디자인
20     NaN    NaN     패션디자인
21     NaN    NaN      섬유예술
22     NaN    NaN      도

In [20]:
table_documents = []

for i, df in enumerate(cleaned_df):
  table_doc = Document(page_content=df.to_markdown(index=False),
                       metadata={'Header 2': '별표'+str(i)})
  table_documents.append(table_doc)

In [21]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)
doc = text_splitter.create_documents([full_text])
simple_vectorstore = FAISS.from_documents(doc, upstage_embeddings)

In [22]:
# 장 내부에 텍스트가 너무 긴 경우 (chunk size 1500 이상) 추가 split
fine_grained_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=100
)

In [23]:
big_docs = []

# 메인 조항 % 부칙 필요 시 split 후 append
for doc in md_header_splits:
    sub_chunks_content = fine_grained_splitter.split_text(doc.page_content)
    for chunk_content in sub_chunks_content:
        # 기존 metadata 유지
        new_metadata = doc.metadata.copy()
        big_docs.append(Document(page_content=chunk_content, metadata=new_metadata))

for table in table_documents:
  big_docs.append(Document(page_content=table.page_content, metadata=table.metadata))

# Now create the vector store from the fine_grained_documents
big_vectorstore = FAISS.from_documents(big_docs, upstage_embeddings)

for doc in big_docs[:3]:
    print(doc.page_content)
    print(doc.metadata)
    print('----------------------')

# 이화여자대학교 학칙  
## 제1장 총칙  
### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에 공헌할 수 있는 지도여성을 양성함을 목적으로 한다.  
### 제2조(명칭)
본교는 이화여자대학교라 부른다.  
### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
{'Header 2': '제1장 총칙'}
----------------------
## 제2장 편제  
### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각 대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다). (개정 2016.6.16., 2017.5.15.)  
### 제5조(학부․학과․전공 및 정원)
① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.4.)
② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할 전공을 결정하게 

In [24]:
small_docs = []

for doc in small_headers_splits:
  new_metadata = doc.metadata.copy()
  small_docs.append(Document(page_content=doc.page_content, metadata=new_metadata))

for table in table_documents:
  small_docs.append(Document(page_content=table.page_content, metadata=table.metadata))

small_vectorstore = FAISS.from_documents(small_docs, upstage_embeddings)

In [25]:
llm = ChatUpstage(api_key = UPSTAGE_API_KEY, model="solar-pro2")

In [26]:
prompt_template = PromptTemplate.from_template(
    """
    You are a quiz taker about Ewha Womans University's regulation.
    Please provide most correct answer for the given context in the response format below.

    Response Format:
    [Answer]: (A) answer <OR> (X) The information is not present in the context.
    [Reason]: Brief reason for your answer
    [Final Answer]: THE final answer to the question following your given reason. Answer in this format: (A) answer

    If the answer is not present in the context, please answer as "(X) The information is not present in the context."

    Example:
    QUESTION) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점은 몇점인가?
    (A) 인정 안됨
    (B) 1학점
    (C) 2학점
    (D) 3학점

    Answer)
    [Answer]: (D) 3학점
    [Reason]: The context explicitly states in 제48조의2(영어 및 정보인증) ①: "영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 이를 각 3학점으로 인정하고 인증서를 교부할 수 있다." This directly answers the question by specifying that 3 credits are recognized for such certifications.
    [Final Answer]: (D) 3학점

    Here is the question and context:
    ---
    Question: {question}
    ---
    Context: {context}
    """
)

In [27]:
df = pd.read_csv(TESTSET_PATH)
prompts = df['prompts']
answers = df['answers']

ewha_prompts = prompts[:25]
ewha_answers = answers[:25]

In [28]:
def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are no answers formatted like the format, it returns None.
    """
    pattern = r"\[Final Answer\]:\s\(([A-J])\)[^)]*"

    # Readjustment로 인해 dinal answer가 여러 번 등장할 수 있으므로 findall 사용
    matches = re.findall(pattern, response)

    if matches:
        print(f'Matched answer: {matches}')  # debug
        # 가장 마지막 final answer만 선택
        return matches[-1].strip()
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        print(f'Re-matched answer: {match}')  # debug
        return match.group(0)
    else:
        return None

In [29]:
def get_responses(qa, prompts):
  responses = []

  for prompt in prompts:
  # for prompt in ewha_prompts:
      response = qa.invoke(prompt)
      # responses.append(response.content)
      print(response['result'])  # debuggiing

      final_answer = extract_answer(response['result'])
      print(final_answer)
      print('************************')
      responses.append(final_answer)

  return responses

In [30]:
def get_accuracy(responses, answers):
  cnt = 0

  for answer, response in zip(answers, responses):
      print("-"*10)
      # generated_answer = extract_answer(response)
      # print(response)
      # check
      if response:
          print(f"generated answer: {response}, answer: {answer}")
      else:
          print(f"extraction fail, answer: {answer}")


      if response == None:
          continue
      if response in answer:
          cnt += 1

  # print()
  # print(f"acc: {(cnt/len(ewha_prompts))*100}%")

  accuracy = (cnt/len(answers))*100
  return accuracy

In [31]:
def majority_voting(qa, prompt):
  majority_voted_answers = []
  num_iterations = 5

  for idx, prompt in enumerate(prompt):
      answers_for_current_prompt = []
      for _ in range(num_iterations):
          response = qa.invoke(prompt)
          final_answer = extract_answer(response['result'])
          if final_answer:
              answers_for_current_prompt.append(final_answer)

      if answers_for_current_prompt:
          answer_counts = collections.Counter(answers_for_current_prompt)
          most_common_answer = answer_counts.most_common(1)[0][0]
          majority_voted_answers.append(most_common_answer)
          print(f"Prompt {idx+1}: Majority voted answer = {most_common_answer}")
      else:
          majority_voted_answers.append(None)
          print(f"Prompt {idx+1}: No valid answers extracted.")

  return majority_voted_answers

In [32]:
big_retriever = big_vectorstore.as_retriever(search_kwargs={"k": 3}) #top 3 문서반환
big_qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=big_retriever,
                                 chain_type="stuff",
                                 chain_type_kwargs={
                                     "prompt":prompt_template,
                                     "document_variable_name": "context",
                                 })

In [33]:
small_retriever = small_vectorstore.as_retriever(search_kwargs={"k": 5}) #top 5 문서반환
small_qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=small_retriever,
                                 chain_type="stuff",
                                 chain_type_kwargs={
                                     "prompt":prompt_template,
                                     "document_variable_name": "context",
                                 })

In [34]:
# 두 retriever의 결과를 ensemble
retriever = EnsembleRetriever(
    retrievers=[big_retriever, small_retriever],
    weights=[0.3, 0.7]    # small chunk의 정확도가 더 높을 때 small에 가중치↑
)

ensemble_qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=retriever,
                                 chain_type="stuff",
                                 chain_type_kwargs={
                                     "prompt":prompt_template,
                                     "document_variable_name": "context",
                                 })

# MMLU_Pro1

In [35]:
llm_classifier = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [36]:
llm_solver = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1,
)

In [37]:
llm_solar_pro = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [38]:
category_index = {
    "law":        "./mmlu_category/law",
    "psychology": "./mmlu_category/psychology",
    "business":   "./mmlu_category/business",
    "philosophy": "./mmlu_category/philosophy",
    "history":    "./mmlu_category/history",
}

categories = list(category_index.keys())

In [39]:
emb = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large-passage")

# 카테고리별 faiss 한번에 로드해서 캐시
vectorstore = {}
for cat, path in category_index.items():
    vectorstore[cat] = FAISS.load_local(
        folder_path=path,
        embeddings=emb,
        allow_dangerous_deserialization=True,
    )

wiki = wikipediaapi.Wikipedia(user_agent= "NLP-RAG/1.0", language = 'en')

In [40]:
from langchain_core.prompts import ChatPromptTemplate
import json

# 질문 분석을 위한 전용 프롬프트
P_ANALYZE_TMPL = """
You are an expert Question Analyst for MMLU-Pro.
Your task is NOT to answer the question, but to dissect it so that a researcher can find the best evidence.

[Available Categories]
- law
- psychology
- business
- philosophy
- history
- other

[Task]
Analyze the question and options to provide a structured JSON output.
You must differentiate between search strategies for **Textbooks** (Vector DB) and **Wikipedia** (Keyword Match).

[Output Fields]
1. "category": Choose ONE from the list above.
2. "core_intent": Summarize what the question is asking in 1 sentence.
3. "constraints": List strict conditions (time period, specific person, negation 'NOT', location).
4. "search_queries": Generate 3-5 **sentence-style** queries for vector search.
   - **CRITICAL**: Translate specific scenarios into **Professional/Legal/Academic Terminology**.
   - Example: "Man hit neighbor" -> "Tort law battery elements"
5. "wiki_keywords": Extract 3-5 **specific entities or nouns** for Wikipedia title search.
   - Example: ["Battery (tort)", "Common law", "Trespass"]

[Example Shot]
Question: "According to Jean Piaget, at which stage of cognitive development do children begin to think logically about abstract concepts and hypothetical situations?"
Options: (A) Sensorimotor (B) Preoperational (C) Concrete operational (D) Formal operational

Output:
{{
  "category": "psychology",
  "core_intent": "Identify the Piagetian stage associated with abstract and hypothetical reasoning.",
  "constraints": ["Jean Piaget's theory", "Abstract concepts", "Hypothetical situations", "Logic"],
  "search_queries": [
    "Piaget cognitive development stages abstract reasoning",
    "Characteristics of formal operational stage",
    "Difference between concrete and formal operational stages"
  ],
  "wiki_keywords": [
    "Jean Piaget",
    "Cognitive development",
    "Formal operational stage",
    "Theory of cognitive development"
  ]
}}

[Real Input]
Question:
{question_text}

Options:
{options_text}

Return ONLY the valid JSON object.
""".strip()

p_analyze_prompt = ChatPromptTemplate.from_template(P_ANALYZE_TMPL)

def run_p_analyze(question_text: str, options: dict, llm=None) -> dict:
    """
    질문을 분석하여 검색 쿼리와 제약 조건을 추출합니다.
    """
    if llm is None:
        llm = llm_solver  # 기존에 정의된 llm 객체 사용

    # 옵션 텍스트화
    options_str = "\n".join([f"({k}) {v}" for k, v in options.items()])

    messages = p_analyze_prompt.format_messages(
        question_text=question_text,
        options_text=options_str
    )

    resp = llm.invoke(messages)
    raw = resp.content.strip()

    # JSON 파싱 (실패 시 기본값 반환 처리 포함)
    try:
        # 마크다운 코드 블록 제거 등 기본적인 정제
        if "```json" in raw:
            raw = raw.split("```json")[1].split("```")[0].strip()
        elif "```" in raw:
            raw = raw.split("```")[0].strip()

        analysis = json.loads(raw)
        return analysis
    except Exception as e:
        print(f"[WARN] p_analyze failed: {e}")
        # 실패 시 기본값: 검색어는 질문 그대로
        return {
            "core_intent": "Unknown",
            "constraints": [],
            "search_queries": [question_text]
        }

In [41]:
def fetch_wiki_context(keywords, window_size=2000):
    snippets = []
    seen_titles = set()

    for kw in keywords:
        kw = (kw or "").strip()
        if not kw: continue

        candidates = [kw, kw.title()]
        # 괄호 제거 등 추가 처리
        if "(" in kw: candidates.append(kw.split("(")[0].strip())

        found_flag = False
        for cand in candidates:
            if cand in seen_titles: continue

            try:
                page = wiki.page(cand)
                if page.exists():
                    seen_titles.add(cand)
                    full_text = page.text

                    # [개선] 단순히 앞부분만 자르는 게 아니라, 키워드가 등장하는 위치를 찾음
                    # 키워드가 텍스트 내에 있으면 그 주변을 가져옴. 없으면 앞부분 가져옴.
                    lower_text = full_text.lower()
                    lower_kw = kw.lower()
                    start_idx = lower_text.find(lower_kw)

                    if start_idx == -1:
                        # 키워드 못 찾으면 그냥 앞부분
                        display_text = full_text[:window_size]
                    else:
                        # 키워드 주변부 (앞으로 500자, 뒤로 1500자 정도)
                        start_pos = max(0, start_idx - 500)
                        end_pos = min(len(full_text), start_idx + window_size)
                        display_text = full_text[start_pos:end_pos]

                    clean_text = display_text.replace('\n', ' ')
                    snippets.append(f"[Wikipedia: {page.title}]\n...{clean_text}...")
                    found_flag = True
                    break
            except Exception:
                continue

    if not snippets:
        return ""

    return "\n\n".join(snippets)

In [42]:
############################################
# 0. 상수 & 기본 설정
############################################

TOP_K_TEXTBOOK = 5
TOP_K_WIKI = 5

In [43]:
############################################
# 1. 질문 + 옵션(A~I) 파싱
############################################

# 1. 질문 + 옵션(A~J) 파싱
def parse_question_and_options(prompt: str):
    """
    prompt: QUESTION + (A) ~ (J) 옵션이 한 문자열에 들어있는 형태
    return: question_text(str), options(dict: 'A'~'J' -> str)
    """
    pattern = r"\(([A-J])\)\s*"
    matches = list(re.finditer(pattern, prompt))

    if not matches:
        return prompt.strip(), {}

    first = matches[0]
    question_text = prompt[:first.start()].strip()

    options: Dict[str, str] = {}
    for idx, m in enumerate(matches):
        letter = m.group(1)  # 'A'~'J'
        start = m.end()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(prompt)
        opt_text = prompt[start:end].strip()
        options[letter] = opt_text

    return question_text, options

In [44]:
############################################
# 2. "Final Answer: X" 파서
############################################

def extract_final_answer_letter(text: str) -> str:
    """
    p_ans 출력에서 'Final Answer: X' 의 X(A~J)를 뽑기.
    """
    m = re.search(r"Final Answer:\s*([A-J])\s*$", text, flags=re.IGNORECASE | re.MULTILINE)
    if m:
        return m.group(1).upper()
    return ""

In [45]:

############################################
# 3. Memory p_gen (INTERNAL 문서 생성)
############################################

P_MEM_TMPL = """
You are a knowledgeable expert in {category}.

Task: Using ONLY your own internal knowledge (without seeing any external documents),
generate a short document that provides accurate and relevant information to help
answer the given exam question.

If you truly do not know the answer or lack enough information, explicitly state
"I don't know" rather than hallucinating facts.

Exam Question:
{question_text}

Write ONE coherent document that:
- Focuses only on concepts, definitions, and relations that are relevant to the question.
- Is self-contained and clear enough to help another model answer the question.
- Avoids unnecessary details.

Document:
""".strip()

p_mem_prompt = ChatPromptTemplate.from_template(P_MEM_TMPL)


def run_p_mem(category: str, question_text: str, llm=None) -> str:
    if llm is None:
        llm = llm_solver
    msgs = p_mem_prompt.format_messages(
        category=category,
        question_text=question_text,
    )
    resp = llm.invoke(msgs)
    return resp.content.strip()

In [46]:
def run_p_gen_for_docs(
    category: str,             # 사용 안 함 (호환성 위해 남겨둠)
    source_type: str,          # "TEXTBOOK" or "WIKIPEDIA"
    question_text: str,        # 사용 안 함 (호환성 위해 남겨둠)
    raw_passages: List[str],   # 검색된 원본 텍스트 리스트
    start_doc_idx: int = 1,
    llm=None,                  # 사용 안 함 (호환성 위해 남겨둠)
) -> Tuple[str, int]:
    """
    [수정됨] LLM을 통한 요약/재작성을 수행하지 않습니다.
    검색된 Raw Passage를 그대로 포맷팅하여 반환합니다.
    이렇게 해야 정보 손실(Information Loss) 없이 p_con이나 p_ans 단계로 넘어갑니다.
    """
    doc_blocks = []
    cur_idx = start_doc_idx

    for k, passage in enumerate(raw_passages, start=1):
        # 빈 텍스트 무시
        if not passage or not passage.strip():
            continue

        # [중요] LLM 호출(invoke) 없이 원본 텍스트를 그대로 넣습니다.
        # 필요하다면 너무 긴 문단만 파이썬 문자열 슬라이싱으로 자르세요 (예: [:2000])
        clean_passage = passage.strip()

        block = (
            f"[Doc {cur_idx} | SOURCE=EXTERNAL({source_type}) | ORIG_ID={source_type}_{k}]\n"
            f"{clean_passage}"
        )
        doc_blocks.append(block)
        cur_idx += 1

    return "\n\n".join(doc_blocks), cur_idx

In [47]:

############################################
# 5. Retrieval helpers (Hybrid: Vector + BM25)
############################################

# 전역 캐시 (BM25 인덱스를 매번 만들지 않기 위해 저장)
bm25_store = {}   # {category: BM25Okapi_object}
doc_store = {}    # {category: [text_list]}

def init_bm25_for_category(category: str):
    """
    [초기화] 해당 카테고리의 모든 문서를 로드하여 BM25 인덱스를 생성
    """
    if category in bm25_store:
        return

    if category not in vectorstore:
        return

    print(f"[Init] Building BM25 index for: {category}...")
    vs = vectorstore[category]

    try:
        # FAISS 메모리에서 모든 문서 텍스트 추출
        # (주의: FAISS 로드 시 데이터가 메모리에 있어야 함)
        all_docs = list(vs.docstore._dict.values())
        texts = [d.page_content for d in all_docs]

        if not texts:
            print(f"[Warn] No texts found in docstore for {category}.")
            return

        # 토크나이징 (단순 띄어쓰기 기준)
        tokenized_corpus = [doc.lower().split() for doc in texts]

        # 저장
        bm25_store[category] = BM25Okapi(tokenized_corpus)
        doc_store[category] = texts
        print(f"[Init] BM25 ready for {category} ({len(texts)} docs)")

    except Exception as e:
        print(f"[Warn] Failed to init BM25 for {category} (Pure Vector mode will be used): {e}")


def reciprocal_rank_fusion(results_list: List[List[str]], k=60) -> List[str]:
    """
    [RRF 알고리즘] 여러 검색 결과의 순위를 합산하여 재정렬
    """
    fused_scores = {}

    for docs in results_list:
        for rank, doc_text in enumerate(docs):
            if doc_text not in fused_scores:
                fused_scores[doc_text] = 0
            # 순위가 높을수록(rank가 작을수록) 높은 점수 부여
            fused_scores[doc_text] += 1 / (rank + k)

    # 점수 높은 순 정렬
    reranked = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    return [text for text, score in reranked]


def get_textbook_passages(category: str, query: str) -> List[str]:
    """
    [수정된 함수] Hybrid Search 적용
    1. Vector Search (의미 검색) -> 10개
    2. BM25 Search (키워드 검색) -> 10개
    3. RRF Fusion -> 상위 5개 반환
    """
    if category not in vectorstore:
        return []

    # 0. BM25 준비
    if category not in bm25_store:
        init_bm25_for_category(category)

    # 1. Vector Search (Semantic)
    vs = vectorstore[category]
    # 후보를 10개 정도 가져옴
    vec_candidates = vs.similarity_search(query, k=10)
    vec_results = [d.page_content for d in vec_candidates]

    # 2. BM25 Search (Keyword) - 만약 준비 안됐으면 생략
    bm25_results = []
    if category in bm25_store:
        bm25 = bm25_store[category]
        docs = doc_store[category]

        tokenized_query = query.lower().split()
        # 키워드 매칭 상위 10개
        bm25_results = bm25.get_top_n(tokenized_query, docs, n=10)

    # 3. Fusion (하나만 있으면 그것만 반환)
    if not bm25_results:
        final_passages = vec_results
    else:
        # 두 결과를 섞음
        final_passages = reciprocal_rank_fusion([vec_results, bm25_results], k=60)

    # 최종 TOP_K 개수만큼 자르기
    return final_passages[:TOP_K_TEXTBOOK]


def get_wiki_passages(keywords: List[str]) -> List[str]:
    """
    (기존 유지)
    """
    try:
        raw = fetch_wiki_context(keywords)
    except Exception as e:
        print(f"[WARN] Wikipedia retrieval failed: {e}")
        return []

    raw = (raw or "").strip()
    if not raw:
        return []
    return [raw]

In [48]:

P_CON_TMPL = """
You are a Knowledge Integrator for MMLU-Pro dataset.

Task: Consolidate information from both your own memorized documents (INTERNAL)
and externally retrieved documents (EXTERNAL) in response to the given exam question.

Guidelines:
* For documents that provide consistent information, cluster them together and
  summarize the key details into a single, concise document.
* For documents with conflicting information, separate them into distinct documents,
  ensuring each document captures the unique perspective or data.
* Exclude any information that is irrelevant to the question.
* For each NEW document you create, clearly indicate:
  - Whether the source was INTERNAL(MEMORY) or EXTERNAL(TEXTBOOK/WIKIPEDIA).
  - The original document numbers that contributed to it (e.g., FROM=1,3,4).

[Examples]

<Example 1: Conflict Resolution>
Initial Context:
[Doc 1 | SOURCE=INTERNAL(MEMORY)]
The Treaty of Versailles was signed in 1918.
[Doc 2 | SOURCE=EXTERNAL(TEXTBOOK)]
The Treaty of Versailles was signed on June 28, 1919, officially ending WWI.

Question: When was the Treaty of Versailles signed?

New Context:
[Doc 1 | SOURCE=EXTERNAL(TEXTBOOK) | FROM=2]
The Treaty of Versailles was signed on June 28, 1919.
(Correction: Internal memory incorrectly stated 1918, which was the armistice year, not the treaty signing.)
</Example 1>

<Example 2: Merging Consistent Info>
Initial Context:
[Doc 1 | SOURCE=EXTERNAL(WIKIPEDIA)]
Classical conditioning involves a neutral stimulus becoming a conditioned stimulus.
[Doc 2 | SOURCE=EXTERNAL(TEXTBOOK)]
Pavlov's dog experiment demonstrated classical conditioning by pairing a bell with food.

Question: Explain the mechanism of classical conditioning.

New Context:
[Doc 1 | SOURCE=MERGED(EXTERNAL) | FROM=1,2]
Classical conditioning is a learning process where a neutral stimulus (like a bell) becomes a conditioned stimulus after being paired with an unconditioned stimulus (like food). This was demonstrated in Pavlov's dog experiment.
</Example 2>

[Real Task]

Initial Context (numbered documents):
{context_init}

Last Context (may be empty):
{last_context}

Question:
{question_text}

Now produce your New Context following the examples.
""".strip()

p_con_prompt = ChatPromptTemplate.from_template(P_CON_TMPL)

def run_p_con(
    question_text: str,
    context_init: str,
    last_context: str = "",
    llm=None,
) -> str:
    if llm is None:
        try:
            llm = llm_solar_pro
        except NameError:
            llm = llm_solver

    msgs = p_con_prompt.format_messages(
        question_text=question_text,
        context_init=context_init,
        last_context=last_context,
    )
    resp = llm.invoke(msgs)
    return resp.content.strip()


In [49]:
############################################
# 7. Knowledge Consolidation + Answer Finalization p_ans (10-Option Optimized)
############################################

import re

def extract_final_answer_letter(text: str) -> str:
    """
    모델이 내뱉는 다양한 형식을 모두 잡아내는 강력한 추출 함수
    """
    if not text: return ""

    # 1순위: "Final Answer" 뒤에 오는 첫 번째 영문자 (A-J) 찾기
    # 예: "**Final Answer:** (A)", "Final Answer: Option A", "Final Answer is [A]"
    # [^\nA-J]* : 줄바꿈이나 A-J가 나오기 전까지의 모든 특수문자(:, *, 공백 등)를 무시
    match = re.search(r"Final Answer[^\nA-J]*([A-J])", text, flags=re.IGNORECASE | re.MULTILINE)
    if match:
        return match.group(1).upper()

    # 2순위: <ANSWER> 태그 (프롬프트에서 지시한 경우)
    match_tag = re.search(r"<ANSWER>\s*\(?([A-J])\)?", text, flags=re.IGNORECASE)
    if match_tag:
        return match_tag.group(1).upper()

    # 3순위: 최후의 수단 - 문장 맨 끝에 있는 (A) 형태 찾기
    fallback = re.findall(r"\(?([A-J])\)?", text.split("\n")[-1])
    if fallback:
        return fallback[-1].upper()

    return "" # 정말 답이 없는 경우


In [50]:
CATEGORY_META = {
    "law": {
        "role": (
            "You are a distinguished Law Professor and Bar Exam Grader. "
            "You specialize in applying strict legal doctrines to complex fact patterns. "
            "Your expertise covers Contracts, Torts, Criminal Law, Property, Constitutional Law, and Evidence."
        ),
        "guidelines": (
            "- Spot the precise legal issue (e.g., specific crime, tort, contract doctrine).\n"
            "- Apply traditional common-law elements unless a statute in the context clearly overrides them.\n"
            "- For crimes, require that ALL elements are satisfied; if one element is missing, that crime is wrong.\n"
            "- For 'best defense' or 'most likely' questions, prefer options that directly attack missing elements or raise strong procedural bars "
            "(e.g., Statute of Frauds, Statute of Limitations)."
        ),
    },
    "business": {
        "role": (
            "You are a CPA (Certified Public Accountant) and Economics Professor. "
            "You excel at financial accounting, managerial accounting, corporate finance, and micro/macroeconomics."
        ),
        "guidelines": (
            "- Carefully extract all numerical data and relationships before computing.\n"
            "- Show each calculation step explicitly (no guessing): check signs, totals, and units.\n"
            "- For finance questions, check if time value of money (PV/FV, discounting) is implied by dates or interest rates.\n"
            "- For conceptual items, ground answers in standard models (supply/demand, elasticity, basic game theory) rather than intuition."
        ),
    },
    "history": {
        "role": (
            "You are an expert Historian specializing in primary source analysis. "
            "You handle World, U.S., European, and Asian history with a focus on chronology and causality."
        ),
        "guidelines": (
            "- First fix the time period and region using names, events, or terminology.\n"
            "- Eliminate options that are anachronistic (wrong century, wrong ruler, wrong war).\n"
            "- Distinguish short-term triggers from long-term structural causes when evaluating explanations.\n"
            "- When reading passages, consider author, audience, and purpose to match the most plausible interpretation."
        ),
    },
    "philosophy": {
        "role": (
            "You are an expert Philosopher specializing in Ethics, Metaphysics, and Epistemology. "
            "You are precise with terminology and familiar with canonical arguments (Kant, Mill, Aristotle, etc.)."
        ),
        "guidelines": (
            "- Identify the relevant school or author first (e.g., Kantian deontology vs. Millian utilitarianism).\n"
            "- Pay close attention to technical terms (e.g., validity vs. truth, necessary vs. sufficient, a priori vs. a posteriori).\n"
            "- Reject options that contradict the core commitments of the view (e.g., utilitarianism ignoring consequences).\n"
            "- Prefer options that match the exact formulation of the principle, not just something that sounds reasonable."
        ),
    },
    "psychology": {
        "role": (
            "You are a Clinical and Research Psychologist. "
            "You specialize in DSM-5 criteria, developmental stages, cognitive psychology, and experimental design."
        ),
        "guidelines": (
            "- For diagnosis, match symptoms strictly to DSM-5 criteria, especially duration, severity, and exclusion conditions.\n"
            "- For development/theory items, map behaviors to the correct named theory and precise stage (e.g., Piaget, Kohlberg, Erikson).\n"
            "- In research design questions, clearly identify IV, DV, and likely confounds; distinguish correlation from causation.\n"
            "- For ethics scenarios, prioritize APA principles such as informed consent, confidentiality, and minimizing harm."
        ),
    },
}

DEFAULT_CATEGORY_META = {
    "role": (
        "You are an expert exam solver for MMLU-Pro. You are careful, analytical, and precise."
    ),
    "guidelines": (
        "- Read the QUESTION carefully and respect all stated constraints.\n"
        "- Use the provided Context as primary evidence and avoid unsupported assumptions.\n"
        "- Eliminate options that conflict with the facts, definitions, or chronology in the Context.\n"
        "- Choose the single best-supported option, even if other options seem partially plausible."
    ),
}

In [51]:
P_ANS_TMPL = """
You are an expert multiple-choice exam solver for MMLU-Pro, using an Astute-RAG style reasoning process.

[Role]
{category_role}

[Category-Specific Guidelines]
{category_guidelines}

You are given:
[Initial Context]  (short, noisy, or partial)
{context_init}

[Consolidated Context]  (higher-quality, filtered evidence)
{context_con}

[QUESTION]
{question_with_options}

Your job is to carefully combine your INTERNAL knowledge with the given EXTERNAL context,
and then follow the **three-stage Astute-RAG reasoning** below.

--------------------------------
Stage 1 – Evidence Aggregation (Astute Step 1)
--------------------------------
1. From the QUESTION ONLY (ignore context for now), extract 3–5 key clues:
   - important concepts, time periods, entities, relationships, or definitions.
2. Using the CONTEXT (Initial + Consolidated), list 3–7 **relevant evidence statements**:
   - each statement should be short (1 sentence),
   - include only facts that help discriminate between the options.
3. If the context contradicts itself, give higher trust to the **Consolidated Context**,
   but never violate explicit constraints stated in the QUESTION.

--------------------------------
Stage 2 – Option-wise Diagnosis with Score & Confidence (Astute Step 2)
--------------------------------
For each option (A, B, C, D, ...), do the following:

1. In 1–2 sentences, compare the option with the key clues + evidence:
   - Explain how it is supported or contradicted.
   - Check whether it satisfies all critical constraints in the QUESTION.

2. Assign three evaluations to this option:
   - A **Label**: one of
     * "CLEARLY SUPPORTED"
     * "PARTIALLY SUPPORTED / DUBIOUS"
     * "CLEARLY CONTRADICTED"
   - A numerical **Score** between 0 and 1:
     * 0.0 ≈ completely wrong, 1.0 ≈ very strongly supported.
   - A short **Confidence** phrase: one of ["high", "medium", "low"].

Use the following format for each option:

Option A:
- Reasoning: ...
- Label: CLEARLY SUPPORTED
- Score: 0.82
- Confidence: high

(Repeat this exact structure for options B, C, D, ...)

--------------------------------
Stage 3 – Global Consistency Check & Final Decision (Astute Step 3)
--------------------------------
1. Look at ALL options, their Labels, and Scores together.
2. Eliminate every option that is:
   - labeled "CLEARLY CONTRADICTED", or
   - has Score < 0.30 (unless every option is very low-scoring).
3. Among the remaining options, choose **one single BEST** answer by asking:
   - Which option is most strongly and directly supported by the evidence and key clues?
   - Does any option rely on extra assumptions not justified by the context?
4. If two options have similar Scores, briefly explain why the higher one
   is still better given the QUESTION wording and constraints.

Finally, on the last line of your response, output the answer in the format:

Final Answer: X

(where X is a single capital letter such as A, B, C, D, ...).
Do NOT output anything after this line.
""".strip()
p_ans_prompt = ChatPromptTemplate.from_template(P_ANS_TMPL)

In [52]:
# 살짝 변경
def run_p_ans(
    context_init: str,
    context_con: str,
    full_question: str,
    category: str,
    llm=None,
) -> Dict[str, Any]:
    """
    p_gen/p_con으로 만들어진 context를 기반으로,
    astute-RAG 방식(INTERNAL vs EXTERNAL, Step 1~4)을 그대로 따르되,
    category별 역할/전략 텍스트를 추가하여 특화된 solver로 동작하게 함.
    """
    if llm is None:
        try:
            llm = llm_solar_pro
        except NameError:
            llm = llm_solver

    meta = CATEGORY_META.get(category, DEFAULT_CATEGORY_META)

    msgs = p_ans_prompt.format_messages(
        category_role=meta["role"],
        category_guidelines=meta["guidelines"],
        context_init=context_init,
        context_con=context_con,
        question_with_options=full_question,
    )
    resp = llm.invoke(msgs)
    raw = resp.content.strip()
    letter = extract_final_answer_letter(raw)
    return {"final_answer": letter, "raw_reasoning": raw}

In [53]:
def solve_mmlu_astute_style(
    full_prompt: str,
    use_wiki: bool = True,
    n_vote: int = 5
) -> Dict[str, Any]:

    # ---------------------------------------------------------
    # 1. 통합 분석 (One-Shot Prompt 적용된 run_p_analyze 사용)
    # ---------------------------------------------------------
    question_text, options = parse_question_and_options(full_prompt)

    try:
        # [변경] 여기서 Category, Textbook Query, Wiki Keywords를 한 번에 다 가져옴
        analysis = run_p_analyze(question_text, options, llm=llm_solver)
    except NameError:
        print("[Error] run_p_analyze not defined. Using fallback.")
        analysis = {}

    # 결과 추출 (안전하게 get 사용)
    # 1) 카테고리 (구버전 함수 classify_... 삭제됨)
    category = analysis.get("category", "business")

    # 2) 제약조건
    constraints = analysis.get("constraints", [])

    # 3) 검색어 분리
    # 교과서용 (문장형)
    textbook_queries = analysis.get("search_queries", [question_text])
    # 위키용 (명사형)
    wiki_keywords = analysis.get("wiki_keywords", [])

    # [Fallback] 만약 위키 키워드가 비었으면 교과서 쿼리라도 씀
    if not wiki_keywords:
        wiki_keywords = textbook_queries

    print(f"   -> [Analysis] Category: {category}")
    print(f"   -> [Textbook Query] {textbook_queries[:1]}...")
    print(f"   -> [Wiki Keywords] {wiki_keywords}")

    # 벡터 DB 확인
    if category not in vectorstore:
        print(f"   -> [Warn] Category '{category}' not found. Fallback to 'business'.")
        category = "business"

    # ---------------------------------------------------------
    # 2. 검색 (Dual-Track: Hybrid Vector + Wiki)
    # ---------------------------------------------------------

    # Track A: Textbook (Vector + BM25 Hybrid)
    # 문장형 쿼리에 제약조건을 더해서 문맥 강화
    combined_textbook_query = " ".join(textbook_queries) + " " + " ".join(constraints)
    tb_passages = get_textbook_passages(category, combined_textbook_query)

    # Track B: Wikipedia (Entity Search)
    # 명사형 키워드로 정확한 표제어 매칭
    wiki_passages = []
    if use_wiki:
        wiki_passages = get_wiki_passages(wiki_keywords)

    # ---------------------------------------------------------
    # 3. Context 생성 (p_mem -> p_gen -> p_con)
    # ---------------------------------------------------------
    doc_blocks = []
    next_doc_idx = 1

    # Internal Memory
    mem_doc_text = run_p_mem(category, question_text)
    if mem_doc_text.strip():
        doc_blocks.append(f"[Doc {next_doc_idx} | SOURCE=INTERNAL(MEMORY)]\n{mem_doc_text}")
        next_doc_idx += 1

    # Textbook Docs (LLM 요약 없이 Pass-through)
    if tb_passages:
        tb_block, next_doc_idx = run_p_gen_for_docs(
            category, "TEXTBOOK", question_text, tb_passages, next_doc_idx, None
        )
        if tb_block.strip(): doc_blocks.append(tb_block)

    # Wiki Docs (LLM 요약 없이 Pass-through)
    if wiki_passages:
        wiki_block, next_doc_idx = run_p_gen_for_docs(
            category, "WIKIPEDIA", question_text, wiki_passages, next_doc_idx, None
        )
        if wiki_block.strip(): doc_blocks.append(wiki_block)

    context_init = "\n\n".join(doc_blocks)

    # Consolidation
    context_con = run_p_con(question_text=question_text, context_init=context_init)

    # ---------------------------------------------------------
    # 4. Voting (Self-Consistency)
    # ---------------------------------------------------------

    # Voting용 질문 구성 (제약조건 명시)
    constraints_str = "\n".join([f"- {c}" for c in constraints])
    augmented_question = f"{full_prompt}\n\n[CRITICAL CONSTRAINTS]\n{constraints_str}"

    # 다양성을 위한 Temperature 설정
    llm_voter = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2", temperature=0.7)

    votes = []
    reasonings = []

    print(f"   -> [Voting] Running {n_vote} iterations...")
    for i in range(n_vote):
        try:
            ans = run_p_ans(
                context_init=context_init,
                context_con=context_con,
                full_question=augmented_question,
                category=category,
                llm=llm_voter
            )
            letter = ans["final_answer"]

            # 유효한 답만 투표
            if letter and letter in "ABCDEFGHIJ":
                votes.append(letter)

            reasonings.append(f"[Run {i+1} Answer: {letter}]\n{ans['raw_reasoning']}")

        except Exception as e:
            print(f"      [Vote Error]: {e}")
            continue

    # 최종 결과 집계
    if not votes:
        final_ans = "A" # Fallback
        vote_summary = "None"
    else:
        # 최빈값 선택
        count = Counter(votes)
        final_ans = count.most_common(1)[0][0]
        vote_summary = str(dict(count))
        print(f"   -> [Result] Winner: {final_ans} | Votes: {vote_summary}")

    full_log = "\n" + "="*40 + "\n".join(reasonings)

    return {
        "final_answer": final_ans,
        "category": category,
        "analysis": analysis,
        "keywords": wiki_keywords, # 변경됨: extracted_keywords -> wiki_keywords
        "context_init": context_init,
        "context_con": context_con,
        "raw_reasoning": full_log,
        "vote_summary": vote_summary
    }

# MMLU_Pro2

In [54]:
category_prompt_template = """
You are an expert exam question category classifier and expert at extracting essential keywords from Question that helps a QA system to retrieve the most relevant context from reference materials.


There are 5 possible categories:
- law
- psychology
- business
- philosophy
- history

[Task]
Given a multiple-choice exam question (including all options),
1. Choose one best category from the list above.
2. Extract exactly 3 important keywords (one noun that consists of single word or short phrase)
   that will be useful to search textbooks and Wikipedia.
   Keywords should be as specific and accurate as possible (e.g., "consent",
   "cognitive dissonance", "Keynesian economics").

[Output format]
Return a valid JSON with the following fields:
- "category": one of ["law","psychology","business","philosophy","history"]
- "keywords": a list of exactly 3 strings

This is an output example:
{{
  "category": "psychology",
  "keywords": ["informed consent", "assent", "child counseling"]
}}

[Question]
{question}
""".strip()

category_prompt = ChatPromptTemplate.from_template(category_prompt_template)

In [55]:
def classify_and_extract_keywords(question_text: str):
    messages = category_prompt.format_messages(question=question_text)
    resp = llm_classifier.invoke(messages)
    raw = resp.content.strip()

    if not raw:
        # 완전 빈 응답이면 기본값으로 대충 처리 (죽지 않게)
        print("[WARN] Empty LLM output for category, fallback to 'history'")
        return "history", ["history", "war", "europe"]

    # 1차 시도: 전체를 JSON으로 해석
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        # 2차 시도: 문자열 안에서 {...} 구간만 잘라서 해석
        start = raw.find("{")
        end = raw.rfind("}")
        if start == -1 or end == -1:
            print("[WARN] No JSON object found in output, fallback to 'history'")
            return "history", ["history", "war", "europe"]

        json_str = raw[start:end+1]
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError as e:
            print("[WARN] JSON parse failed again:", e)
            return "history", ["history", "war", "europe"]

    category = data.get("category", "history").strip().lower()
    keywords = data.get("keywords", [])
    keywords = [str(k).strip() for k in keywords][:3]

    if not keywords:
        keywords = ["history", "war", "europe"]

    return category, keywords

In [56]:
def fetch_wiki_context(keywords, max_chars_per_page=1200):
    snippets = []
    for kw in keywords:
        try:
            page = wiki.page(kw)
            if not page.exists():
                print(f"[WARN] page for '{kw}' does not exist")
                continue
            text = page.text[:max_chars_per_page]
            snippets.append(f"[Wikipedia: {page.title}]\n{text}")
        except Exception as e:
            print(f"[WARN] wikipediaapi fetch failed for '{kw}': {e}")
            continue

    return "\n\n".join(snippets)

In [57]:
TOP_K_TEXTBOOK = 5

def build_context_for_solver(question_text: str, category: str, keywords):
    # 1) 카테고리별 textbook RAG
    vs = vectorstore[category]
    docs = vs.similarity_search(question_text, k=TOP_K_TEXTBOOK)
    textbook_context = "\n\n".join(
        f"[Textbook Doc {i+1}] {d.page_content}" for i, d in enumerate(docs)
    )

    # 2) Wikipedia context
    wiki_context = fetch_wiki_context(keywords)

    full_context = f"""\
=== TEXTBOOK CONTEXT ({category}) ===
{textbook_context}

=== WIKIPEDIA CONTEXT (keywords: {', '.join(keywords)}) ===
{wiki_context}
"""
    return full_context

In [58]:
SOLVER_PROMPT_LAW_TMPL = """
You are a U.S. law professor helping a legal QA system retrieve the most relevant authority.

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

Use the CONTEXT as textbook + Wikipedia support, but ALWAYS follow these principles:

1) PRIORITIZE THE QUESTION FACT PATTERN
- First, read the QUESTION (including facts and answer options) very carefully.
- Treat the facts and wording in the QUESTION as the primary authority.
- Do NOT contradict the facts as stated in the QUESTION.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY LEGAL CLUES FROM THE QUESTION
Before judging the options, silently identify 3–5 essential legal clues from the QUESTION itself, such as:
- relevant area of law (e.g., common-law robbery, larceny, attempt, accomplice liability, defenses),
- key facts about timing, knowledge, intent (mens rea), consent, force, or possession,
- jurisdiction assumptions (assume standard U.S. common law / majority rule unless the CONTEXT clearly says otherwise).

Base your reasoning on these clues.

3) USE CONTEXT CAREFULLY (TEXTBOOK > WIKIPEDIA)
- Use the CONTEXT to recall or confirm black-letter rules, elements, and standard doctrines.
- If textbook material and Wikipedia conflict, trust the textbook-style explanation in the CONTEXT.
- Do NOT narrow or expand a rule beyond what is normally accepted in standard common law / majority doctrine,
  unless the CONTEXT explicitly tells you to use a specific variant.

4) APPLY BLACK-LETTER LAW PRECISELY
- Identify the relevant rule(s) (e.g., elements of common-law larceny, robbery, burglary, receiving stolen property,
  attempt, conspiracy, self-defense, etc.).
- Apply every element to the facts in the QUESTION.
- Pay careful attention to:
  - timing of force or threats,
  - whether the defendant knew property was stolen,
  - whether there was a trespassory taking,
  - whether possession was obtained by fraud, trick, or without consent,
  - whether any required mental state (intent, knowledge, recklessness, negligence) is clearly present.

5) COMPARE EACH OPTION STRICTLY
For EACH answer choice:
- Explain briefly (in your own reasoning) why it is correct or incorrect,
  always referencing specific facts from the QUESTION and the applicable legal rule.
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the precise legal definition and all elements,
    - correctly reflects the timing and mental state required by the rule,
    - does NOT add extra facts not in the QUESTION,
    - fits both the QUESTION facts and the CONTEXT.

6) AVOID HALLUCINATIONS
- Do NOT assume extra premises, background facts, or historical claims that are not stated or strongly implied.
- Do NOT change the content of a doctrine to make an option fit better; respect standard textbook interpretations.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) CONSISTENCY CHECK BETWEEN REASONING AND OPTIONS
Before you output the final answer, you MUST do a short consistency check:
- First, clearly state in one sentence what you believe the correct answer is
  (for numerical questions, state the exact number; for conceptual questions, state the exact statement or rule).
- Then, choose an option ONLY if it exactly matches that stated conclusion.
- If none of the options exactly match your stated conclusion, you MUST:
  1) re-check your reasoning or calculation once, and
  2) update your conclusion so that it matches a standard rule or formula from the CONTEXT,
     and then select the option that exactly matches this corrected conclusion.
- You are NOT allowed to pick an option just because it is “closest” to your conclusion.


8) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences.
- Write the source you retrived specifically
- Do NOT use Markdown formatting (no bullet points, no headings, no **bold**, no code blocks).
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_law = ChatPromptTemplate.from_template(SOLVER_PROMPT_LAW_TMPL)

In [59]:
SOLVER_PROMPT_PHILOSOPHY_TMPL = """
You are an expert philosophy exam solver. You specialize in advanced multiple-choice questions
(undergraduate and graduate level) in ethics, meta-ethics, epistemology, metaphysics, logic,
philosophy of mind and language, political philosophy, and the history of philosophy
(ancient, medieval, modern, and contemporary).

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

The CONTEXT acts like textbook + Wikipedia support. The QUESTION is the primary source.
Follow these rules very carefully:

1) PRIORITIZE THE QUESTION TEXT
- First, read the QUESTION (including any passage and all answer options) very carefully.
- Treat the text and wording of the QUESTION as the strongest evidence.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY PHILOSOPHICAL CLUES FROM THE QUESTION
When solving the problem, follow this procedure:
1. Look only at the [QUESTION] text first and silently extract 3–5 essential philosophical clues, such as:
   - the main concept (e.g., categorical imperative, utilitarianism, internalism vs externalism,
     Gettier problem, a priori vs a posteriori, necessary vs sufficient conditions),
   - the relevant philosopher or school (e.g., Kant, Mill, Hume, Plato, Aristotle, Rawls, behaviorism, logical positivism),
   - the logical structure (e.g., what follows from what, which statement would they reject, which principle is violated).
2. Then, look at these clues and the CONTEXT and compare them.
   - If the logic between the QUESTION clues and the CONTEXT conflicts, trust the QUESTION clues.
3. If the clues within textbook-style parts of the CONTEXT and Wikipedia-style parts are conflicted,
   trust the textbook-style exposition in the CONTEXT.

3) USE CONTEXT TO RECALL DEFINITIONS, NOT TO OVERRIDE THE QUESTION
- Use the CONTEXT to recall standard definitions, distinctions, and canonical views of philosophers.
- Do NOT invent new doctrines or attribute views to philosophers that are not well-established.
- Apply the standard reading unless the CONTEXT explicitly specifies a different interpretation.

4) APPLY PHILOSOPHICAL DOCTRINES PRECISELY
- Identify which theory, argument, or distinction the question is really testing.
- Pay attention to:
  - necessary vs sufficient conditions,
  - analytic vs synthetic, a priori vs a posteriori,
  - validity vs soundness,
  - deontological vs consequentialist reasoning,
  - internal vs external justification,
  - the exact wording of a principle or formulation (e.g., Kant’s Humanity formulation, Mill’s harm principle).
- For “Which of the following would X most likely endorse/reject?” style questions, base your choice on
  X’s actual doctrine, not on your own judgment of what is true.

5) COMPARE EACH OPTION CAREFULLY
For EACH option:
- Briefly evaluate why it could be correct or incorrect, using explicit references to:
  - the key clues you extracted from the QUESTION, and
  - the relevant definitions or doctrines from the CONTEXT.
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the canonical definition or the philosopher’s standard view,
    - captures the fundamental idea of the theory rather than a peripheral detail,
    - does not add extra assumptions not stated in the QUESTION,
    - uses the correct logical strength (e.g., does not turn “some” into “all,” does not confuse necessary with sufficient).

6) AVOID HALLUCINATIONS
- Do NOT assume extra premises, background facts, or historical claims that are not stated or strongly implied.
- Do NOT change the content of a doctrine to make an option fit better; respect standard textbook interpretations.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) CONSISTENCY CHECK BETWEEN REASONING AND OPTIONS
Before you output the final answer, you MUST do a short consistency check:

- First, clearly state in one sentence what you believe the correct answer is
  (for numerical questions, state the exact number; for conceptual questions, state the exact statement or rule).
- Then, choose an option ONLY if it exactly matches that stated conclusion.
- If none of the options exactly match your stated conclusion, you MUST:
  1) re-check your reasoning or calculation once, and
  2) update your conclusion so that it matches a standard rule or formula from the CONTEXT,
     and then select the option that exactly matches this corrected conclusion.
- You are NOT allowed to pick an option just because it is “closest” to your conclusion.

8) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences (no bullet points, no headings, no Markdown formatting like ** or #).
- Be concise but logically clear.
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_philosophy = ChatPromptTemplate.from_template(SOLVER_PROMPT_PHILOSOPHY_TMPL)

In [60]:
SOLVER_PROMPT_PSYCHOLOGY_TMPL = """
You are an expert psychology exam solver. You specialize in advanced multiple-choice questions
(undergraduate and graduate level) in clinical psychology, cognitive psychology, social psychology,
developmental psychology, personality, biological/physiological psychology, learning, psychometrics,
research methods, and ethics (e.g., APA code of conduct).

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

The CONTEXT acts like textbook + Wikipedia support. The QUESTION is the primary source.
Follow these rules very carefully:

1) PRIORITIZE THE QUESTION TEXT
- First, read the QUESTION (including any vignette, description, and all answer options) very carefully.
- Treat the facts and wording in the QUESTION as the strongest evidence.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY PSYCHOLOGICAL CLUES FROM THE QUESTION
When solving the problem, follow this procedure:
1. Look only at the [QUESTION] text first and silently extract 3–5 essential psychological clues, such as:
   - the main construct or theory (e.g., classical conditioning, operant conditioning, working memory,
     attachment style, Big Five traits, self-efficacy, cognitive dissonance),
   - the relevant theorist or model (e.g., Piaget, Vygotsky, Erikson, Kohlberg, Bandura, Beck),
   - the developmental stage or level (e.g., preconventional vs conventional, sensorimotor vs formal operational),
   - the type of design or method (e.g., experiment vs correlational study, longitudinal vs cross-sectional),
   - the key symptoms, behaviors, or ethical requirements described in the vignette.
2. Then, look at these clues and the CONTEXT and compare them.
   - If the logic between the QUESTION clues and the CONTEXT conflicts, trust the QUESTION clues.
3. If the clues within textbook-style parts of the CONTEXT and Wikipedia-style parts are conflicted,
   trust the textbook-style exposition in the CONTEXT.

3) USE CONTEXT TO RECALL DEFINITIONS, NOT TO OVERRIDE THE QUESTION
- Use the CONTEXT to recall standard definitions, diagnostic criteria (at a conceptual level),
  major theories, stages, and classical experiments.
- Do NOT invent new constructs or attribute theories to psychologists who are not associated with them.
- Apply the standard textbook interpretation unless the CONTEXT explicitly specifies a different one.

4) APPLY PSYCHOLOGICAL THEORIES PRECISELY
- Identify which theory, construct, stage, or principle the question is really testing.
- Pay attention to:
  - necessary features vs incidental details of a concept,
  - differences between similar concepts (e.g., negative reinforcement vs punishment,
    state vs trait, reliability vs validity),
  - differences between stages (e.g., Kohlberg’s Stage 1 vs Stage 2, Erikson’s crises),
  - the direction of cause/effect or prediction implied by the theory.
- For ethics questions (e.g., APA), carefully consider:
  - informed consent, confidentiality, competence, dual relationships, and risk of harm,
  - what the code obligates the psychologist to do in the specific situation.

5) COMPARE EACH OPTION CAREFULLY
For EACH answer choice:
- Briefly evaluate why it could be correct or incorrect, using explicit references to:
  - the key clues you extracted from the QUESTION, and
  - the relevant definitions or theories from the CONTEXT.
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the canonical textbook definition or theory,
    - directly fits ALL key facts given in the QUESTION (age, setting, behavior, stated intentions),
    - does not rely on extra assumptions not stated in the QUESTION,
    - has the correct level of generality (not too narrow, not too broad) for the construct or stage.

6) AVOID HALLUCINATIONS
- Do NOT assume additional symptoms, diagnoses, or background history that are not clearly stated.
- Do NOT assume the person has specific knowledge or intentions beyond what the QUESTION describes.
- Do NOT guess rare or exotic explanations when a standard, well-known theory clearly fits better.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) CONSISTENCY CHECK BETWEEN REASONING AND OPTIONS
Before you output the final answer, you MUST do a short consistency check:

- First, clearly state in one sentence what you believe the correct answer is
  (for numerical questions, state the exact number; for conceptual questions, state the exact statement or rule).
- Then, choose an option ONLY if it exactly matches that stated conclusion.
- If none of the options exactly match your stated conclusion, you MUST:
  1) re-check your reasoning or calculation once, and
  2) update your conclusion so that it matches a standard rule or formula from the CONTEXT,
     and then select the option that exactly matches this corrected conclusion.
- You are NOT allowed to pick an option just because it is “closest” to your conclusion.

8) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences (no bullet points, no headings, no Markdown formatting like ** or #).
- Be concise but logically clear.
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_psychology = ChatPromptTemplate.from_template(SOLVER_PROMPT_PSYCHOLOGY_TMPL)

In [61]:
SOLVER_PROMPT_BUSINESS_TMPL = """
You are an expert business and economics exam solver. You specialize in advanced multiple-choice questions
(undergraduate and MBA level) in microeconomics, macroeconomics, finance, accounting, management,
organizational behavior, marketing, operations, business ethics, and quantitative methods.

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

The CONTEXT acts like textbook + Wikipedia support. The QUESTION is the primary source.
Follow these rules very carefully:

1) PRIORITIZE THE QUESTION TEXT
- First, read the QUESTION (including any tables, numbers, and all answer options) very carefully.
- Treat the facts, numerical data, and wording in the QUESTION as the strongest evidence.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY BUSINESS / ECON CLUES FROM THE QUESTION
When solving the problem, follow this procedure:
1. Look only at the [QUESTION] text first and silently extract 3–5 essential clues, such as:
   - the main domain (e.g., microeconomics, macroeconomics, corporate finance, accounting, management, marketing),
   - key concepts (e.g., opportunity cost, elasticity, present value, NPV, CAPM, break-even point,
     comparative advantage, marginal cost/benefit, principal–agent problem, game theory, externalities),
   - relevant formulas or relationships (e.g., PV = CF/(1+r)^t, profit = TR – TC, elasticity definitions,
     accounting identities, basic statistics),
   - organizational/management concepts (e.g., leadership style, motivation theory, decision-making biases),
   - business ethics or corporate governance principles.
2. Then, look at these clues and the CONTEXT and compare them.
   - If the logic between the QUESTION clues and the CONTEXT conflicts, trust the QUESTION clues.
3. If the clues within textbook-style parts of the CONTEXT and Wikipedia-style parts are conflicted,
   trust the textbook-style exposition in the CONTEXT.

3) USE CONTEXT TO RECALL DEFINITIONS AND FORMULAS, NOT TO OVERRIDE THE QUESTION
- Use the CONTEXT to recall standard definitions, formulas, and conceptual relationships.
- Do NOT invent new theories or assume non-standard definitions.
- Apply the standard textbook interpretation unless the CONTEXT explicitly specifies a different one.

4) APPLY BUSINESS / ECONOMIC LOGIC PRECISELY
- Identify which concept, model, or calculation the question is really testing.
- Pay careful attention to:
  - the difference between average vs marginal, stock vs flow, nominal vs real, short run vs long run,
  - risk vs return, cost of capital, discounting vs compounding,
  - accounting distinctions (asset/liability/equity, revenue vs profit, cash vs accrual),
  - supply–demand shifts vs movement along a curve,
  - game-theoretic reasoning (dominant strategies, Nash equilibrium),
  - basic statistical reasoning (mean, variance, correlation vs causation).
- When numbers are given, use them logically and consistently.

5) COMPARE EACH OPTION CAREFULLY
For EACH answer choice:
- Briefly evaluate why it could be correct or incorrect, using explicit references to:
  - the key clues you extracted from the QUESTION, and
  - the relevant definitions, models, or formulas from the CONTEXT.
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the exact definitions and standard formulas,
    - correctly uses all given information in the QUESTION (including signs, units, and constraints),
    - does not rely on extra assumptions not stated in the QUESTION,
    - has the correct economic or managerial interpretation (no confusion of cause and effect, or average vs marginal).

6) AVOID HALLUCINATIONS
- Do NOT assume additional data (e.g., extra numbers, extra constraints) that are not given.
- Do NOT assume a different market structure, time period, or financial environment unless clearly stated.
- Do NOT change definitions (e.g., of elasticity, NPV, beta, ROI) to force an option to fit.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) CONSISTENCY CHECK BETWEEN REASONING AND OPTIONS
Before you output the final answer, you MUST do a short consistency check:

- First, clearly state in one sentence what you believe the correct answer is
  (for numerical questions, state the exact number; for conceptual questions, state the exact statement or rule).
- Then, choose an option ONLY if it exactly matches that stated conclusion.
- If none of the options exactly match your stated conclusion, you MUST:
  1) re-check your reasoning or calculation once, and
  2) update your conclusion so that it matches a standard rule or formula from the CONTEXT,
     and then select the option that exactly matches this corrected conclusion.
- You are NOT allowed to pick an option just because it is “closest” to your conclusion.

8) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences (no bullet points, no headings, no Markdown formatting like ** or #).
- Be concise but logically clear.
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_business = ChatPromptTemplate.from_template(SOLVER_PROMPT_BUSINESS_TMPL)

In [62]:
SOLVER_PROMPT_HISTORY_TMPL = """
You are an expert history exam solver. You specialize in advanced multiple-choice questions
(undergraduate and graduate level) in world history, U.S. history, European history, Asian history,
Latin American and African history, intellectual and cultural history, political history,
economic history, and questions based on primary-source excerpts.

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

The CONTEXT acts like textbook + Wikipedia support. The QUESTION is the primary source.
Follow these rules very carefully:

1) PRIORITIZE THE QUESTION TEXT
- First, read the QUESTION (including any passage, quotation, data, and all answer options) very carefully.
- Treat the facts, wording, and time/place cues in the QUESTION as the strongest evidence.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY HISTORICAL CLUES FROM THE QUESTION
When solving the problem, follow this procedure:
1. Look only at the [QUESTION] text first and silently extract 3–5 essential historical clues, such as:
   - time period or approximate dates (e.g., “late 18th century,” “interwar period,” “Post–World War II”),
   - geographic region or polity (e.g., Tang China, Mughal India, Weimar Germany, Cold War United States),
   - type of source (e.g., political speech, treaty, law code, memoir, propaganda, religious text),
   - key themes (e.g., imperialism, industrialization, nationalism, revolution, reform, decolonization, globalization),
   - actors and relationships (e.g., state vs. frontier peoples, colonizer vs. colonized, elite vs. peasantry).
2. Then, look at these clues and the CONTEXT and compare them.
   - If the logic between the QUESTION clues and the CONTEXT conflicts, trust the QUESTION clues.
3. If the clues within textbook-style parts of the CONTEXT and Wikipedia-style parts are conflicted,
   trust the textbook-style exposition in the CONTEXT.

3) USE CONTEXT TO RECALL HISTORY, NOT TO OVERRIDE THE QUESTION
- Use the CONTEXT to recall standard chronology, major events, and characteristic policies or ideas
  of states, movements, and historical figures.
- Do NOT overwrite or ignore specific details given in the QUESTION (e.g., who is speaking, to whom, and in what setting).
- Apply widely accepted historical interpretations unless the CONTEXT or QUESTION clearly specifies a particular viewpoint.

4) INTERPRET SOURCES AND CAUSALITY CAREFULLY
- For questions based on a passage or source:
  - Focus on what the author explicitly states and what is strongly implied by the text.
  - Distinguish between what the author approves of and what they criticize.
  - Note the tone, audience, and purpose (e.g., justify a policy, criticize a ruler, mobilize support).
- Pay careful attention to:
  - cause vs. effect (do not confuse consequences with causes),
  - continuity vs. change over time,
  - similarities vs. differences between regions or periods,
  - whether the question is asking for context, consequence, motivation, or historical significance.

5) COMPARE EACH OPTION CAREFULLY
For EACH answer choice:
- Briefly evaluate why it could be correct or incorrect, using explicit references to:
  - the key clues you extracted from the QUESTION, and
  - relevant facts or patterns from the CONTEXT (chronology, policies, institutions, conflicts).
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the specific time, place, and actors indicated in the QUESTION,
    - fits the main theme or process the QUESTION is testing,
    - does not rely on anachronistic assumptions or events from a different period,
    - avoids overgeneralization or mixing different regions or eras.

6) AVOID HALLUCINATIONS
- Do NOT introduce additional events, dates, or policies that are not supported by the QUESTION or reliable CONTEXT.
- Do NOT move events to different centuries or regions just to make an option fit.
- Do NOT attribute quotes or ideas to the wrong person or regime.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) CONSISTENCY CHECK BETWEEN REASONING AND OPTIONS
Before you output the final answer, you MUST do a short consistency check:
- First, clearly state in one sentence what you believe the correct answer is
  (for numerical questions, state the exact number; for conceptual questions, state the exact statement or rule).
- Then, choose an option ONLY if it exactly matches that stated conclusion.
- If none of the options exactly match your stated conclusion, you MUST:
  1) re-check your reasoning or calculation once, and
  2) update your conclusion so that it matches a standard rule or formula from the CONTEXT,
     and then select the option that exactly matches this corrected conclusion.
- You are NOT allowed to pick an option just because it is “closest” to your conclusion.

8) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences (no bullet points, no headings, no Markdown formatting like ** or #).
- Be concise but logically clear.
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_history = ChatPromptTemplate.from_template(SOLVER_PROMPT_HISTORY_TMPL)

In [63]:
def solve_mmlu_with_rag_and_wiki(question_text: str):
    final_letter = None
    # 1) 1단계 LLM: 카테고리 + 키워드
    category, keywords = classify_and_extract_keywords(question_text)
    if category not in vectorstore:
        print(f"[WARN] Unknown category '{category}', fallback to 'business'")
        category = "business"

    # 2) 컨텍스트 구성 (FAISS + Wikipedia)
    context = build_context_for_solver(question_text, category, keywords)

    # 3) 2단계 LLM: CoT로 정답 도출
    if category == "law":
      messages = solver_prompt_law.format_messages(
          context=context,
          question=question_text
      )
    elif category == "philosophy":
      messages = solver_prompt_philosophy.format_messages(
          context=context,
          question=question_text,
      )
    elif category == "psychology":
      messages = solver_prompt_psychology.format_messages(
          context=context,
          question=question_text,
      )
    elif category == "business":
      messages = solver_prompt_business.format_messages(
          context=context,
          question=question_text,
      )
    elif category == "history":
        messages = solver_prompt_history.format_messages(
          context=context,
          question=question_text,
      )

    resp = llm_solver.invoke(messages)
    raw = resp.content.strip()

    # (1) 아래쪽 줄부터 "FINAL ANSWER"가 들어간 줄 찾기 (마크다운 포함 허용)
    for line in raw.splitlines()[::-1]:  # 아래에서부터 검색
        s = line.strip()

        # '**Final Answer: A**' 같이 생긴 것도 잡기 위해 'in' 사용
        if "FINAL ANSWER" in s.upper():
            # 마크다운 볼드 제거
            s = s.replace("**", "").strip()
            # 정규식으로 'Final Answer: X'에서 X만 뽑기
            m = re.search(r"FINAL ANSWER\s*:\s*([A-Z])", s, re.IGNORECASE)
            if m:
                final_letter = m.group(1).upper()
                break

    # 그래도 못 찾으면, 전체 텍스트에서 대문자 한 글자 검색
    if final_letter is None:
        for ch in raw:
            if "A" <= ch <= "Z":
                final_letter = ch
                break

    return {
        "category": category,
        "keywords": keywords,
        "raw_reasoning": raw,       # 디버깅/분석용
        "final_answer": final_letter,
    }

## mmlu용 majority voting 함수

In [64]:
from collections import Counter
from typing import Callable, Dict, Any

def majority_vote_mmlu(
    solve_once_fn: Callable[[str], Dict[str, Any]],
    prompt: str,
    n_vote: int = 5,
) -> Dict[str, Any]:
    """
    solve_once_fn: full_prompt(str) -> {"final_answer": "A", ...} 형태로 결과를 주는 함수
    prompt: 문제 전체 문자열 (질문 + 보기)
    n_vote: 몇 번 반복할지 (기본 5번)

    return:
        {
          "final_answer": "A",
          "all_votes": ["A","B","A","A","C"],
          "vote_summary": {"A":3, "B":1, "C":1},
          "winner_run": <solve_once_fn가 반환한 dict 중에서 우승 답과 같은 걸 하나>
        }
    """
    votes = []
    outs  = []

    for _ in range(n_vote):
        out = solve_once_fn(prompt)
        outs.append(out)
        ans = (out.get("final_answer") or "").strip().upper()
        if ans and ans in "ABCDEFGHIJ":
            votes.append(ans)

    if not votes:
        # 진짜로 답을 하나도 못 뽑으면 fallback
        return {
            "final_answer": "",
            "all_votes": [],
            "vote_summary": {},
            "winner_run": outs[0] if outs else {},
        }

    counter = Counter(votes)
    final_ans, _ = counter.most_common(1)[0]

    # 우승 답과 같은 run 하나 골라서 reasoning/log 같이 넘겨주기
    winner_run = next((o for o in outs if (o.get("final_answer") or "").upper() == final_ans), outs[0])

    return {
        "final_answer": final_ans,
        "all_votes": votes,
        "vote_summary": dict(counter),
        "winner_run": winner_run,
    }

# 테스트

In [65]:
def is_korean(text: str) -> bool:
    """문자열에 한글이 하나라도 들어있으면 한국어라고 간주."""
    if not isinstance(text, str):
        return False
    return bool(re.search(r"[가-힣]", text))

In [66]:
def solve_ewha_single(prompt: str,
                      qa_chain,
                      n_vote: int = 5) -> str:
    votes = []

    for _ in range(n_vote):
        response = qa_chain.invoke(prompt)   # {'result': '...', 'source_documents': [...]}
        text = response["result"]
        ans = extract_answer(text)          # 네가 위에 정의한 extract_answer 사용
        if ans:
            votes.append(ans.strip().upper())

    if not votes:
        return ""   # 진짜 아무것도 못 뽑으면 빈 문자열

    counter = Counter(votes)
    final_ans, _ = counter.most_common(1)[0]
    return final_ans

In [67]:
def solve_mmlu_pro1_single(prompt: str, outer_votes: int = 5) -> Dict[str, Any]:
    def _solve_once(q: str) -> Dict[str, Any]:
        # 내부 self-consistency는 끄고(=1), 외부에서만 5번 투표
        out = solve_mmlu_astute_style(
            full_prompt=q,
            use_wiki=True,
            n_vote=1,
        )
        # solve_mmlu_astute_style이 이미 "final_answer" 키를 갖고 있다고 가정
        return out

    mv = majority_vote_mmlu(_solve_once, prompt, n_vote=outer_votes)

    winner = mv["winner_run"]
    final_ans = mv["final_answer"]

    return {
        "final_answer": final_ans,
        "category": winner.get("category", ""),
        "raw_reasoning": winner.get("raw_reasoning", ""),
        "vote_summary": mv["vote_summary"],
    }


In [68]:
def solve_mmlu_pro2_single(prompt: str, outer_votes: int = 5) -> Dict[str, Any]:
    def _solve_once(q: str) -> Dict[str, Any]:
        out = solve_mmlu_with_rag_and_wiki(q)
        return out

    mv = majority_vote_mmlu(_solve_once, prompt, n_vote=outer_votes)

    winner = mv["winner_run"]
    final_ans = mv["final_answer"]

    return {
        "final_answer": final_ans,
        "category": winner.get("category", ""),
        "raw_reasoning": winner.get("raw_reasoning", ""),
        "vote_summary": mv["vote_summary"],
    }
    # return final_ans

In [69]:
QUESTION_COL = "prompts"
df = pd.read_csv(TESTSET_PATH)

pred_lang  = []   # 'ko' or 'en'
pred_m1    = []   # (ewha + mmlu_pro1) 정답
pred_m2    = []   # (ewha + mmlu_pro2) 정답

for i, row in df.iterrows():
    q = str(row[QUESTION_COL])
    print(f"[{i}] Solving: {q[:60]}...")

    if is_korean(q):
        # =======================
        # 한국어 → Ewha 학칙 RAG
        # =======================
        pred_lang.append("ko")

        ans_ko = solve_ewha_single(
            prompt=q,
            qa_chain=ensemble_qa,
            n_vote=5,
        )

        # 한국어 문제는 두 버전 CSV 모두 Ewha 답을 사용
        pred_m1.append(ans_ko)
        pred_m2.append(ans_ko)

    else:
        # =======================
        # 영어 → mmlu_pro1 / mmlu_pro2
        # =======================
        pred_lang.append("en")

        # mmlu_pro1: Astute-RAG + 5번 voting
        out1 = solve_mmlu_pro1_single(q, outer_votes=5)
        # final_answer만 쓰고 싶으면 이렇게:
        ans1 = out1["final_answer"] if isinstance(out1, dict) else out1
        pred_m1.append(ans1)

        # mmlu_pro2: category RAG + 5번 voting
        out2 = solve_mmlu_pro2_single(q, outer_votes=5)
        ans2 = out2["final_answer"] if isinstance(out2, dict) else out2
        pred_m2.append(ans2)

# 길이 체크 (디버깅용)
print("len(df)   =", len(df))
print("len(pred_lang) =", len(pred_lang))
print("len(pred_m1)   =", len(pred_m1))
print("len(pred_m2)   =", len(pred_m2))

# -------------------------
# (ewha + mmlu_pro1) 정답 csv
# -------------------------
df_m1 = df.copy()
df_m1["pred_lang"]      = pred_lang
df_m1["rag_cot_answer"] = pred_m1
df_m1.to_csv("testset_ewha_mmlu1.csv", index=False)
print("[Saved] testset_ewha_mmlu1.csv")

# -------------------------
# (ewha + mmlu_pro2) 정답 csv
# -------------------------
df_m2 = df.copy()
df_m2["pred_lang"]      = pred_lang
df_m2["rag_cot_answer"] = pred_m2
df_m2.to_csv("testset_ewha_mmlu2.csv", index=False)
print("[Saved] testset_ewha_mmlu2.csv")

[0] Solving: QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 휴학을 신청하야하나요?
(...
Matched answer: ['D']
Matched answer: ['D']
Matched answer: ['D']
Matched answer: ['D']
Matched answer: ['D']
[1] Solving: QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 의하여 제적된 자는 제적된...
Matched answer: ['A']
Matched answer: ['D', 'A', 'A']
Matched answer: ['D']
Matched answer: ['D', 'D']
Matched answer: ['D', 'A']
[2] Solving: QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 몇학점 이상 ...
Matched answer: ['C']
Matched answer: ['C']
Matched answer: ['C']
Matched answer: ['C']
Matched answer: ['C']
[3] Solving: QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?
(A) 팜 : 징계에 의해 퇴...
Matched answer: ['D']
Matched answer: ['D']
Matched answer: ['D']
Matched answer: ['D']
Matched answer: ['D']
[4] Solving: QUESTION5) 2019학년도 휴먼기계바이오공학부의 입학 정원은 몇 명인가? 
(A) 90명 
(B) 1...
Matched answer: ['C']
Matched answer: ['C']
Matched answer: ['C']
Matched answer: ['C']
Matched answer: ['C']
[5] Solving: QUESTION6) 1980학년도 이전 입학생에 대하여 적

In [70]:
ANSWER_COL = "answers"
ground_truth = df[ANSWER_COL]

acc_m1 = get_accuracy(df_m1["rag_cot_answer"], ground_truth)
acc_m2 = get_accuracy(df_m2["rag_cot_answer"], ground_truth)

print(f"Model 1 Accuracy: {acc_m1:.4f}")
print(f"Model 2 Accuracy: {acc_m2:.4f}")

----------
generated answer: D, answer: (D)
----------
generated answer: A, answer: (A)
----------
generated answer: C, answer: (C)
----------
generated answer: D, answer: (D)
----------
generated answer: C, answer: (C)
----------
generated answer: A, answer: (B)
----------
generated answer: D, answer: (D)
----------
generated answer: C, answer: (C)
----------
generated answer: C, answer: (C)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: C, answer: (C)
----------
generated answer: C, answer: (C)
----------
generated answer: A, answer: (A)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: D, answer: (D)
----------
generated answer: D, answer: (D)
----------
generated answer: B, answer: (D)
----------
generated answer: B, answer: (B)
----------
generated answer: A, answer: (A)
----------
generated answer: D, 